In [1]:
import os, sys
import numpy as np
import h5netcdf

import xarray as xr
import xarray.ufuncs as xu
import pandas as pd

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

plt.rc("font", size=14)
ccrs.PlateCarree()

from dask.distributed import Client, LocalCluster
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4)
client = Client(cluster)
client


/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 47701 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:47701/status,
Dashboard: http://137.129.155.67:47701/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: inproc://137.129.155.67/55947/1,Workers: 1
Dashboard: http://137.129.155.67:47701/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: inproc://137.129.155.67/55947/4,Total threads: 4
Dashboard: http://137.129.155.67:40371/status,Memory: 78.61 GiB
Nanny: None,


In [2]:
latm, latM = -15.1,15.1

def computeProbabilite(latm, latM, ds_skew_mean, lonm = None, lonM = None):
    if lonm != None :
        _ds = ds.sel(lat = slice(latM,latm), lon = slice(lonm,lonM)).persist()
        _ds_anom = ds_anom.sel(lat = slice(latM,latm), lon = slice(lonm,lonM)).persist()
    else :
        _ds = ds.sel(lat = slice(latM,latm)).persist()
        _ds_anom = ds_anom.sel(lat = slice(latM,latm)).persist()
    
    _ds = _ds.sum(dim = ['lat','lon']).compute()
    _ds = _ds.sel(nRange = slice(-10,10))
    
    _ds_anom = _ds_anom.sum(dim = ['lat','lon']).compute()
    _ds_anom = _ds_anom.sel(nRange = slice(-30,30))
    
    Prob = _ds.copy()
    Prob = Prob*0.
    del Prob['time']

    for v in var :
        da = _ds[v]
        Somme = 0
        for i in range(da.nRange.size):
            Somme = Somme + int(da.isel(nRange = i).values)

        Prob[v] = da/Somme

    del Prob['time']

    Prob_T = _ds_anom.copy()
    Prob_T = Prob_T*0.
    # del Prob_T['time']
    Somme = 0
    for i in range(_ds_anom.nRange.size):
        Somme = Somme + int(_ds_anom['tcwv_ano'].isel(nRange = i).values)

    Prob_T['tcwv_ano'] = _ds_anom['tcwv_ano']/Somme

    plt.figure(figsize = (12,10))
    for v in var :
        Prob[v].plot(label = v + ' skewness = ' + str('%.2f' % ds_skew_mean[v]), ylim = [0,0.15])
    plt.legend()
    plt.xlabel('Anomalie')
    plt.ylabel('Probabilité')
    plt.axvline(x = 0, color = 'black' )
    plt.title('Densité de Probabilité pour chaque ondes')
    plt.grid()
    plt.show()
    
    plt.figure(figsize = (12,10))
    Prob_T['tcwv_ano'].plot(label = 'anomalie' + ' skewness = ' + str('%.2f' % ds_skew_mean['tcwv_ano']), ylim = [0,0.050])
    plt.legend()
    plt.xlabel('Anomalie')
    plt.ylabel('Probabilité')
    plt.axvline(x = 0, color = 'black' )
    plt.title('Densité de Probabilité pour l anomalie totale')
    plt.grid()
    plt.show()

In [3]:
indir_data_FILTERED = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/ANALYSIS/DISTRIBUTION/TCWV/'

ds_anom = xr.open_mfdataset(indir_data_FILTERED + 'distribution_tcwv_ano_JAS.nc')
ds_anom = ds_anom.rename({'latitude':'lat','longitude':'lon'})

ds = xr.open_mfdataset(indir_data_FILTERED + 'distribution_TCWV_MJO_JAS.nc')
# da_EIG = xr.open_mfdataset(indir_data_FILTERED + 'distribution_TCWV_EIG.nc')
# da_WIG = xr.open_mfdataset(indir_data_FILTERED + 'distribution_TCWV_WIG.nc')
da_MRG = xr.open_mfdataset(indir_data_FILTERED + 'distribution_TCWV_MRG_JAS.nc')
da_Kelvin = xr.open_mfdataset(indir_data_FILTERED + 'distribution_TCWV_Kelvin_JAS.nc')
da_Rossby = xr.open_mfdataset(indir_data_FILTERED + 'distribution_TCWV_Rossby_JAS.nc')
da_TD = xr.open_mfdataset(indir_data_FILTERED + 'distribution_TCWV_TD_JAS.nc')

# ds['TCWV_EIG'] = da_EIG['TCWV_EIG']
# ds['TCWV_WIG'] = da_WIG['TCWV_WIG']
ds['TCWV_MRG'] = da_MRG['TCWV_MRG']
ds['TCWV_Kelvin'] = da_Kelvin['TCWV_Kelvin']
ds['TCWV_Rossby'] = da_Rossby['TCWV_Rossby']
ds['TCWV_TD'] = da_TD['TCWV_TD']

# ds = ds.squeeze('variable')
var = ['TCWV_MRG','TCWV_Kelvin','TCWV_Rossby','TCWV_TD','TCWV_MJO']

# ds = ds.squeeze('variable')
var = ['TCWV_MRG','TCWV_Kelvin','TCWV_Rossby','TCWV_TD','TCWV_MJO']
ds = ds.assign_coords(lon = (((ds.lon + 180) % 360) - 180)).sortby('lon')
ds = ds.assign_coords(nRange = (ds.nRange + 0.25/2))

ds

<xarray.Dataset>
Dimensions:      (nRange: 241, lat: 41, lon: 180)
Coordinates:
  * nRange       (nRange) float32 -29.88 -29.62 -29.38 ... 29.62 29.88 30.12
    time         datetime64[ns] ...
  * lat          (lat) float64 40.0 38.0 36.0 34.0 ... -34.0 -36.0 -38.0 -40.0
  * lon          (lon) float32 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
Data variables:
    TCWV_MJO     (nRange, lat, lon) float32 dask.array<chunksize=(241, 41, 180), meta=np.ndarray>
    TCWV_MRG     (nRange, lat, lon) float32 dask.array<chunksize=(241, 41, 180), meta=np.ndarray>
    TCWV_Kelvin  (nRange, lat, lon) float32 dask.array<chunksize=(241, 41, 180), meta=np.ndarray>
    TCWV_Rossby  (nRange, lat, lon) float32 dask.array<chunksize=(241, 41, 180), meta=np.ndarray>
    TCWV_TD      (nRange, lat, lon) float32 dask.array<chunksize=(241, 41, 180), meta=np.ndarray>

In [27]:
from scipy.stats import skew
ds_skew = ds.reduce(skew,dim=('time'))

<xarray.Dataset>
Dimensions:      (lat: 41, lon: 180)
Coordinates:
    time         datetime64[ns] 1991-07-01
  * lat          (lat) float64 40.0 38.0 36.0 34.0 ... -34.0 -36.0 -38.0 -40.0
  * lon          (lon) float32 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
Data variables:
    TCWV_MJO     (lat, lon) float32 44.28 44.28 44.28 ... 44.28 44.28 44.28
    TCWV_MRG     (lat, lon) float32 44.28 44.28 44.28 ... 44.28 44.28 44.28
    TCWV_Kelvin  (lat, lon) float32 44.28 44.28 44.28 ... 44.28 44.28 44.28
    TCWV_Rossby  (lat, lon) float32 44.28 44.28 44.28 ... 44.28 44.28 44.28
    TCWV_TD      (lat, lon) float32 44.28 44.28 44.28 ... 44.28 44.28 44.28

# Pour la zone 1 
## lat = 5 °N à 15 °N et lon = 5 °W à 5°E

In [ ]:
latm, latM = 5,15
lonm, lonM = -5, 5
computeProbabilite(latm,latM, ds_skew_mean, lonm = lonm, lonM = lonM)

# Pour la zone 2
## lat = 0 °N à 20 °N et lon = 180 °E à 270 °E

In [ ]:
latm, latM = 0,20
lonm, lonM = 180, 270
computeProbabilite(latm,latM, lonm = lonm, lonM = lonM)

# Pour la zone 3
## lat = 0 °N à 20 °N et lon = 270 °E à 360 °E

In [ ]:
latm, latM = 0,20
lonm, lonM = 270, 360
computeProbabilite(latm,latM, lonm = lonm, lonM = lonM)

# Pour la zone 3
## lat = 0 °N à 20 °N et lon = 0 °E à 100 °E

In [ ]:
latm, latM = 0,20
lonm, lonM = 0, 100
computeProbabilite(latm,latM, lonm = lonm, lonM = lonM)

# Pour la zone 4
## lat = 0 °S à 20 °S et lon = 0 °E à 135 °E

In [ ]:
latm, latM = -20,0
lonm, lonM = 0, 135
computeProbabilite(latm,latM, lonm = lonm, lonM = lonM)

# Pour la zone 5
## lat = 0 °S à 20 °S et lon = 135 °E à 360 °E

In [ ]:
latm, latM = -20,0
lonm, lonM = 40, 90
computeProbabilite(latm,latM, lonm = lonm, lonM = lonM)

# Suite pour les valeurs Brut

In [ ]:
ds_BRUT = xr.open_mfdataset(indir_data_FILTERED + 'distribution_BRUT_tcwv_JAS.nc')
ds_BRUT =ds_BRUT.assign_coords(longitude = (((ds_BRUT.longitude + 180) % 360) - 180)).sortby('longitude')


In [ ]:
ds_BRUT = ds_BRUT.sel(latitude = slice(15,5), longitude = slice(-5,5)).persist()
_ds_BRUT = ds_BRUT.sum(dim = ['latitude','longitude']).compute()

Prob_BRUT = _ds_BRUT.copy()
Prob_BRUT = Prob_BRUT * 0.
# del Prob_T['time']
Somme = 0
for i in range(Prob_BRUT.nRange.size):
    Somme = Somme + int(_ds_BRUT['tcwv'].isel(nRange = i).values)

Prob_BRUT['tcwv'] = _ds_BRUT['tcwv']/Somme

plt.figure(figsize = (12,10))
Prob_BRUT['tcwv'].plot(label = 'Anomalie brut', ylim = [0,0.05])
# _ds_anom['Sum_FILTER'].plot(label = 'Somme Filtre')
plt.legend()
plt.xlabel('Anomalie')
plt.ylabel('Probabilité')
plt.title('Densité de Probabilité pour la collone d eau')
plt.grid()
plt.show()

In [ ]:
ds_BRUT['tcwv'].isel(latitude = 0, longitude = 0).values